In [1]:
from pathlib import Path
pladb = Path('/projects/ps-yeolab5/hsher/jarrett_sf3b/rmats/DMSO-DMSO_DMSO-PladB')
wx23 = Path('/projects/ps-yeolab5/hsher/jarrett_sf3b/rmats/DMSO-DMSO_DMSO-WX-2-23')
wx43 = Path('/projects/ps-yeolab5/hsher/jarrett_sf3b/rmats/DMSO-DMSO_DMSO-WX-2-43')
ddxko = Path('/projects/ps-yeolab5/hsher/jarrett_sf3b/rmats/DMSO-DMSO_dTAGv1-DMSO')

In [2]:
import sys
sys.path.append('/home/hsher/projects/splice_snake/scripts/')
from rmats_grouping import *

In [3]:
def enough_covered_event(df, cov_cutoff = 100):
    # find highly covered
    df['SAMPLE1_read'] = df['IJC_SAMPLE_1'].map(comma_delim_mean)+df['SJC_SAMPLE_1'].map(comma_delim_mean)
    df['SAMPLE2_read'] = df['IJC_SAMPLE_2'].map(comma_delim_mean)+df['SJC_SAMPLE_2'].map(comma_delim_mean)
    qualified_id = df.loc[(df['SAMPLE1_read']>cov_cutoff)&(df['SAMPLE2_read']>cov_cutoff), 'unique_id']
    return qualified_id

In [4]:
def make_unique_id(df, columns = ['chr','exonStart_0base',
       'exonEnd', 'upstreamES', 'upstreamEE', 'downstreamES', 'downstreamEE']):
    df['unique_id'] = df[columns].apply(lambda x: ':'.join(
    [str(i) for i in x]), axis = 1)
def find_subset_events(folder1, folder2, 
                       event = 'A3SS',
                      columns = ['chr', 'strand', 'longExonStart_0base',
       'longExonEnd', 'shortES', 'shortEE','flankingES', 'flankingEE']):
    pos1, neg1, control1 = make_upon_KD_file(folder1/ f'{event}.MATS.JC.txt', min_reads = 100)
    pos2, neg2, control2 = make_upon_KD_file(folder2/ f'{event}.MATS.JC.txt', min_reads = 100)
    
    _ = [make_unique_id(df, columns = columns) for df in [pos1, neg1, control1, pos2, neg2, control2]]
    
    # filter
    pos1 = pos1.loc[pos1['unique_id'].isin(enough_covered_event(pos1))]
    pos2 = pos2.loc[pos2['unique_id'].isin(enough_covered_event(pos2))]
    neg1 = neg1.loc[neg1['unique_id'].isin(enough_covered_event(neg1))]
    neg2 = neg2.loc[neg2['unique_id'].isin(enough_covered_event(neg2))]
    
    # make intersections
    bothpos = pos1.loc[pos1['unique_id'].isin(pos2['unique_id'])]
    pos1_uniq = pos1.loc[~pos1['unique_id'].isin(pos2['unique_id'])]
    pos2_uniq = pos2.loc[~pos2['unique_id'].isin(pos1['unique_id'])]
    
    bothneg = neg1.loc[neg1['unique_id'].isin(neg2['unique_id'])]
    neg1_uniq = neg1.loc[~neg1['unique_id'].isin(neg2['unique_id'])]
    neg2_uniq = neg2.loc[~neg2['unique_id'].isin(neg1['unique_id'])]
    
    both_ctrl = control1.loc[control1['unique_id'].isin(control2['unique_id'])]
         
    return bothpos, pos1_uniq, pos2_uniq, bothneg, neg1_uniq, neg2_uniq, both_ctrl
    

In [5]:
def save_rmats_format(df, fname):
    df.drop(['unique_id', 'SAMPLE1_read','SAMPLE2_read'], axis = 1, inplace = True)
    df.to_csv(fname, sep = '\t')

In [6]:
for event, column in zip(['SE', 'RI', 'MXE', 'A5SS', 'A3SS'],
                        [
                        ['chr','exonStart_0base',
       'exonEnd', 'upstreamES', 'upstreamEE', 'downstreamES', 'downstreamEE'],
                         ['chr', 'strand', 'riExonStart_0base',
       'riExonEnd', 'upstreamES', 'upstreamEE', 'downstreamES', 'downstreamEE'],
                         ['chr', 'strand', '1stExonStart_0base',
       '1stExonEnd', '2ndExonStart_0base', '2ndExonEnd', 'upstreamES',
       'upstreamEE', 'downstreamES', 'downstreamEE'],
                         ['chr', 'strand', 'longExonStart_0base',
       'longExonEnd', 'shortES', 'shortEE','flankingES', 'flankingEE'],
                         ['chr', 'strand', 'longExonStart_0base',
       'longExonEnd', 'shortES', 'shortEE','flankingES', 'flankingEE']
                        ]
                        
                        ):
    bothpos, pos1_uniq, pos2_uniq, bothneg, neg1_uniq, neg2_uniq, both_ctrl=find_subset_events(
        wx23, pladb, event = event, columns = column)
    
    for suffix, df in zip(['bothpos', 'pos1', 'pos2', 'bothneg', 'neg1', 'neg2', 'ctrl'],
                         [bothpos, pos1_uniq, pos2_uniq, bothneg, neg1_uniq, neg2_uniq, both_ctrl]
                         ):
        print(df.shape, event, suffix)
        save_rmats_format(df, Path('plad_wx_compare')/f'{event}.{suffix}.tsv')
    

Going from 424444 events to 75470 significant events.
Number of included upon kd: 51029
Number of excluded upon kd: 8677
Going from 456408 events to 82965 significant events.
Number of included upon kd: 55942
Number of excluded upon kd: 9536
(13116, 25) SE bothpos
(3729, 25) SE pos1
(6233, 25) SE pos2
(919, 25) SE bothneg
(459, 25) SE neg1
(869, 25) SE neg2
(150889, 25) SE ctrl
Going from 36755 events to 9139 significant events.
Number of included upon kd: 930
Number of excluded upon kd: 5040
Going from 38326 events to 9604 significant events.
Number of included upon kd: 981
Number of excluded upon kd: 5165
(154, 25) RI bothpos
(114, 25) RI pos1
(121, 25) RI pos2
(1624, 25) RI bothneg
(583, 25) RI neg1
(758, 25) RI neg2
(13605, 25) RI ctrl
Going from 336241 events to 41351 significant events.
Number of included upon kd: 16050
Number of excluded upon kd: 18069
Going from 402823 events to 49336 significant events.
Number of included upon kd: 19298
Number of excluded upon kd: 21566
(2131,

In [7]:
save_rmats_format(bothpos)

TypeError: save_rmats_format() missing 1 required positional argument: 'fname'

In [ ]:
bothpos['ID.1']

In [ ]:
bothpos.head()

In [ ]:
bothpos.shape

In [ ]:
pos1_uniq.shape

In [ ]:
pos2_uniq.shape

In [ ]:
bothneg.shape

In [ ]:
neg1_uniq.shape

In [ ]:
neg2_uniq.shape

In [ ]:
both_ctrl.shape

In [ ]:
df = pd.read_csv('/home/krothamel/projects/Skipper_pipeline/output_MJ_100522/Metadensity_MJ_100622.csv')

In [ ]:
df['bam_0'] = df['bam_0'].str.replace('/home/krothamel/projects/Skipper_pipeline/output_MJ_100522/output/bams/dedup/genome/',
                       '/home/hsher/scratch/jarrett_sf3b/').values
df['bam_control_0'] = df['bam_control_0'].str.replace('/home/krothamel/projects/Skipper_pipeline/output_MJ_100522/output/bams/dedup/genome/',
                       '/home/hsher/scratch/jarrett_sf3b/').values

In [ ]:
cols = ['minus_0', 'minus_control_0', 'plus_0', 'plus_control_0']
for col in cols:
    df[col] = df[col].str.replace('/home/krothamel/projects/Skipper_pipeline/output_MJ_100522/output/', 
                         '/home/hsher/scratch/jarrett_sf3b/').values

In [ ]:
df.to_csv('temp_eclip.csv')

In [ ]:
import os
df.applymap(os.path.isfile)

In [ ]:
[Path(p).name for p in df['bam_0']]

In [ ]:
[Path(p).parent for p in df['bam_0']]